In [ ]:
#path settings
import os
drive_path = "/content/drive/MyDrive/cv_final"
data_path = "data/testing"
output_path = "output/testing"

#first task 0_center_frame
task_1_dir = os.path.join(drive_path,data_path,"0_center_frame")
task_1_output = os.path.join(drive_path,output_path,"0_center_frame")

# second task 1_30fps_to_240fps
task_2_dir = os.path.join(drive_path,data_path,"1_30fps_to_240fps")
task_2_output = os.path.join(drive_path,output_path,"1_30fps_to_240fps")

# third task 2_24fps_to_60fps
task_3_dir = os.path.join(drive_path,data_path,"2_24fps_to_60fps")
task_3_output = os.path.join(drive_path,output_path,"2_24fps_to_60fps")

In [ ]:
#確認GPU
import torch
print(torch.cuda.is_available())
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

True
name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB


In [ ]:
#GMA package
!git clone https://github.com/brianchung0803/GMA.git
!pip install einops
%cd GMA/core/

# Environment settings
import sys

sys.path.append('core')

import argparse
import cv2
import glob
import numpy as np
import imageio
import matplotlib.pyplot as plt

from network import RAFTGMA
from utils import flow_viz
from utils.utils import InputPadder
from skimage import data,exposure,img_as_float
from PIL import Image,ImageFilter

DEVICE = 'cuda'

def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)


def viz(img, flo, flow_dir):
    img = img[0].permute(1, 2, 0).cpu().numpy()
    flo = flo[0].permute(1, 2, 0).cpu().numpy()

    # map flow to rgb image
    flo = flow_viz.flow_to_image(flo)
    cv2_imshow(flo)

    imageio.imwrite(os.path.join(flow_dir, 'flo.png'), flo)
    print(f"Saving optical flow visualisation at {os.path.join(flow_dir, 'flo.png')}")


def normalize(x):
    return x / (x.max() - x.min())


args = {'model': '/content/GMA/checkpoints/gma-things.pth',
        'model_name': 'GMA',
        'path': '/content/drive/MyDrive/cv_final_project/data/data/validation/0_center_frame/5/input',
        'num_heads': 1,
        'position_only': False,
        'position_and_content': False,
        'mixed_precision': False}


def get_optflow_GMA(img0_path, img1_path, show=False):
    args = {'model': '/content/GMA/checkpoints/gma-things.pth', 
            'model_name': 'GMA',
            'num_heads': 1,
            'position_only': False,
            'position_and_content': False,
            'mixed_precision': False}
    
    model = torch.nn.DataParallel(RAFTGMA(args))
    model.load_state_dict(torch.load(args['model']))

    model = model.module
    model.to(DEVICE)
    model.eval()

    with torch.no_grad():
        img0 = load_image(img0_path)
        img1 = load_image(img1_path)
        ###sharpen###
        #kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
        #img0 = torch.FloatTensor(  np.transpose(cv2.filter2D(np.array(PIL.Image.open(img0_path)), -1, kernel),[2,0,1])[np.newaxis,]  ).cuda()
        #img1 = torch.FloatTensor(  np.transpose(cv2.filter2D(np.array(PIL.Image.open(img1_path)), -1, kernel),[2,0,1])[np.newaxis,]  ).cuda()
        ###sharpen###
        b, c, h, w = img0.shape
        padder = InputPadder(img0.shape)
        img0, img1 = padder.pad(img0, img1)
        flow_low, flow_up = model(img0, img1, iters=12, test_mode=True)
        flow_up = flow_up[0].permute(1, 2, 0).cpu().numpy()
        flow_up = cv2.resize(flow_up, (w, h))
        
        if show:
            flow_up = flow_viz.flow_to_image(flow_up)
            cv2_imshow(flow_up)

        # 回傳optical flow
        return flow_up

%cd ../../

fatal: destination path 'GMA' already exists and is not an empty directory.
/content/GMA/core
/content


In [ ]:
#backward warping
from torch.autograd import Variable
class backWarp(torch.nn.Module):  
    def __init__(self):
        pass
        
    def forward(self, x, flo, boolv):
        B, C, H, W = x.size()
        # mesh grid 
        xx = torch.arange(0, W).view(1,-1).repeat(H,1)
        yy = torch.arange(0, H).view(-1,1).repeat(1,W)
        xx = xx.view(1,1,H,W).repeat(B,1,1,1)
        yy = yy.view(1,1,H,W).repeat(B,1,1,1)
        grid = torch.cat((xx,yy),1).float()
       
        #x = x.cuda()
        #grid = grid.cuda()
        vgrid = Variable(grid) + flo # B,2,H,W

        vgrid[:,0,:,:] = 2.0*vgrid[:,0,:,:].clone()/max(W-1,1)-1.0 
        vgrid[:,1,:,:] = 2.0*vgrid[:,1,:,:].clone()/max(H-1,1)-1.0

        
        vgrid = vgrid.permute(0,2,3,1)#from B,2,H,W -> B,H,W,2
        mask = torch.autograd.Variable(torch.ones(x.size()))
        if boolv == 0:
          output = torch.nn.functional.grid_sample(x, vgrid,align_corners=True)
          mask = torch.nn.functional.grid_sample(mask, vgrid,align_corners=True)
        else:
          output = torch.nn.functional.grid_sample(x, vgrid,align_corners=True,mode='bicubic')
          mask = torch.nn.functional.grid_sample(mask, vgrid,align_corners=True,mode='bicubic')          

        mask[mask<0.9999] = 0
        mask[mask>0] = 1

        return output*mask

In [ ]:
#API
def origin(F01, F10, t):
  return (1-t)*F01

def approx0(F01, F10, t): #Ft0
  #t = 1-t
  return -(1-t)*t*F01 + t*t*F10

def approx1(F01, F10, t): #Ft1
  return (1-t)*(1-t)*F01 - t*(1-t)*F10

def approxMask(F01, F10, occ_mask_F01, occ_mask_F10, t):
  F01 *= np.concatenate([occ_mask_F01[np.newaxis,],occ_mask_F01[np.newaxis,]], axis=0)[np.newaxis,]
  F10 *= np.concatenate([occ_mask_F10[np.newaxis,],occ_mask_F10[np.newaxis,]], axis=0)[np.newaxis,]
  mask = occ_mask_F01 * occ_mask_F10
  maskinv = 1 - mask
  return ((F01+F10)*maskinv + ((1-t)*(1-t)*F01 - t*(1-t)*F10)*mask).float()

def approxMaskOnlyFilter(F01, F10, occ_mask_F01, occ_mask_F10, t):
  F01 *= np.concatenate([occ_mask_F01[np.newaxis,],occ_mask_F01[np.newaxis,]], axis=0)[np.newaxis,]
  F10 *= np.concatenate([occ_mask_F10[np.newaxis,],occ_mask_F10[np.newaxis,]], axis=0)[np.newaxis,]
  return ((1-t)*(1-t)*F01 - t*(1-t)*F10).float()

def integrateIt(I01,I10,img0,img1,t):
  I01 = np.transpose(I01.squeeze().numpy(),(1,2,0))
  I10 = np.transpose(I10.squeeze().numpy(),(1,2,0))
  It = np.zeros(I01.shape).astype(I01.dtype)
  
  It[I01==0] = I10[I01==0]
  It[I10==0] = I01[I10==0]
  It[np.logical_and(I01!=0,I10!=0)] = ((1-t)*I10+t*I01)[np.logical_and(I01!=0,I10!=0)]

  img0 = np.transpose(img0.squeeze().numpy(), (1,2,0))
  img1 = np.transpose(img1.squeeze().numpy(), (1,2,0))
  
  
  if t<0.4:
    It[It==0] = img0[It==0]
  elif t>0.6:
    It[It==0] = img1[It==0]
  else:
    It[It==0] = ((img0+img1)/2)[It==0]
  
  return It
  

def VFI_function(arugments_strFirst, arugments_strSecond, arugments_strPosition, arugments_strOutput, type):
  first = os.path.join(arugments_strPosition,arugments_strFirst)
  second = os.path.join(arugments_strPosition,arugments_strSecond)
  
  '''
  kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
  tenFirst = torch.FloatTensor(np.ascontiguousarray(np.array(PIL.Image.open(first)))[:, :, ::-1].transpose(2, 0, 1).astype(np.float32) * (1.0 / 255.0))
  tenSecond = torch.FloatTensor(numpy.ascontiguousarray(np.array(PIL.Image.open(first)))[:, :, ::-1].transpose(2, 0, 1).astype(numpy.float32) * (1.0 / 255.0))
  tenOutput = estimate(tenFirst, tenSecond)
  tenOutput_2 = estimate(tenSecond, tenFirst)
  F01 = tenOutput[np.newaxis,:,:,:]
  F10 = tenOutput_2[np.newaxis,:,:,:]
  '''
  #cv2.filter2D(np.array(PIL.Image.open(first)),-1,kernel)
  #cv2.filter2D(np.array(PIL.Image.open(first)),-1,kernel)

  F01 = torch.FloatTensor(np.transpose(get_optflow_GMA(first,second),(2,0,1))[np.newaxis])
  F10 = torch.FloatTensor(np.transpose(get_optflow_GMA(second,first),(2,0,1))[np.newaxis])

  img0 = cv2.imread(first)  
  img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB).astype(np.int32)
  img0 = torch.FloatTensor(np.transpose(img0,(2,0,1))[np.newaxis,:,:,:])

  img1 = cv2.imread(second) 
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB).astype(np.int32)
  img1 = torch.FloatTensor(np.transpose(img1,(2,0,1))[np.newaxis,:,:,:])

  #occ_mask_F01, occ_mask_F10 = get_occ_mask(np.transpose(F01.squeeze().numpy(),[1,2,0]), np.transpose(F10.squeeze().numpy(),[1,2,0]))
  #F01 *= np.concatenate([occ_mask_F01[np.newaxis,],occ_mask_F01[np.newaxis,]], axis=0)[np.newaxis,]
  #F10 *= np.concatenate([occ_mask_F10[np.newaxis,],occ_mask_F10[np.newaxis,]], axis=0)[np.newaxis,]

  backwarp = backWarp()
  if(type==1.0):
    print("1st ok")
    t=0.5
    I01 = backwarp.forward(img1, approx1(F01,F10,t).float(), 1)
    I10 = backwarp.forward(img0, approx0(F01,F10,t).float(), 1) ###approx(F10,F01,t)
    test = integrateIt(I01,I10,img0,img1,t)
    cv2.imwrite(os.path.join(arugments_strOutput,'frame10i11.jpg'), cv2.cvtColor(test,cv2.COLOR_RGB2BGR))
  elif(type==2.0):
    idx = int(arugments_strFirst.lstrip('input/').rstrip('.jpg'))
    print("2nd ok with", idx)
    for i in range(1,8):
      t=i/8
      I01 = backwarp.forward(img1, approx1(F01,F10,t).float(), 0)
      I10 = backwarp.forward(img0, approx0(F01,F10,t).float(), 0)
      test = integrateIt(I01,I10,img0,img1,t)
      cv2.imwrite(os.path.join(arugments_strOutput,f'{(idx+i):05d}'+'.jpg'), cv2.cvtColor(test,cv2.COLOR_RGB2BGR))
  elif(type==3.0):
    idx = int(arugments_strFirst.lstrip('input/').rstrip('.jpg'))
    print("3rd ok with", idx)
    t=0.4
    I01 = backwarp.forward(img1, approx1(F01,F10,t).float(), 0)
    I10 = backwarp.forward(img0, approx0(F01,F10,t).float(), 0)
    test = integrateIt(I01,I10,img0,img1,t)
    cv2.imwrite(os.path.join(arugments_strOutput,f'{(idx+4):05d}'+'.jpg'), cv2.cvtColor(test,cv2.COLOR_RGB2BGR))
    t=0.8
    I01 = backwarp.forward(img1, approx1(F01,F10,t).float(), 0)
    I10 = backwarp.forward(img0, approx0(F01,F10,t).float(), 0)
    test = integrateIt(I01,I10,img0,img1,t)
    cv2.imwrite(os.path.join(arugments_strOutput,f'{(idx+8):05d}'+'.jpg'), cv2.cvtColor(test,cv2.COLOR_RGB2BGR))
  elif(type==4.0):
    idx = int(arugments_strFirst.lstrip('input/').rstrip('.jpg'))
    print("4th ok with", idx)
    t=0.2
    I01 = backwarp.forward(img1, approx1(F01,F10,t).float(), 0)
    I10 = backwarp.forward(img0, approx0(F01,F10,t).float(), 0)
    test = integrateIt(I01,I10,img0,img1,t)
    cv2.imwrite(os.path.join(arugments_strOutput,f'{(idx+2):05d}'+'.jpg'), cv2.cvtColor(test,cv2.COLOR_RGB2BGR))
    t=0.6
    I01 = backwarp.forward(img1, approx1(F01,F10,t).float(), 0)
    I10 = backwarp.forward(img0, approx0(F01,F10,t).float(), 0)
    test = integrateIt(I01,I10,img0,img1,t)
    cv2.imwrite(os.path.join(arugments_strOutput,f'{(idx+6):05d}'+'.jpg'), cv2.cvtColor(test,cv2.COLOR_RGB2BGR))
  else:
    print("error")

In [ ]:
# #testing
from pathlib import Path

for i in range(7,17):
  dir = os.path.join(task_1_dir,"{}".format(i)) #'1_30fps_to_240fps/3/0/'
  out =  os.path.join(task_1_output,"{}".format(i))  #'1_30fps_to_240fps/3/0/'
  Path(out).mkdir(parents=True, exist_ok=True)
  first = 'input/frame10.png'              #'0_center_frame/7/input/frame10.png'
  second = 'input/frame11.png'             #'0_center_frame/7/input/frame11.png'
  VFI_function(first, second, dir, out, 1)

#output file with '1_30fps_to_240fps/'


for i in range(3,5):
  for j in range(12):
    dir = os.path.join(task_2_dir,"{}".format(i),"{}".format(j)) #'1_30fps_to_240fps/3/0/'
    out =  os.path.join(task_2_output,"{}".format(i),"{}".format(j))  #'1_30fps_to_240fps/3/0/'
    Path(out).mkdir(parents=True, exist_ok=True)
    first = 'input/' + f'{(8*j):05d}' + '.jpg'       #'1_30fps_to_240fps/3/0/input/00000.jpg'
    second = 'input/' + f'{(8*(j+1)):05d}' + '.jpg'     #'1_30fps_to_240fps/3/0/input/00008.jpg'
    VFI_function(first, second, dir, out, 2)

#output file with 2_24fps_to_60fps/'
for i in range(3,5):
  for j in range(8):
    dir = os.path.join(task_3_dir,"{}".format(i),"{}".format(j)) #'1_30fps_to_240fps/3/0/'
    out =  os.path.join(task_3_output,"{}".format(i),"{}".format(j))  #'1_30fps_to_240fps/3/0/'
    Path(out).mkdir(parents=True, exist_ok=True)
    first = 'input/' + f'{(10*j):05d}' + '.jpg'       #'1_30fps_to_240fps/3/0/input/00000.jpg'
    second = 'input/' + f'{(10*(j+1)):05d}' + '.jpg'     #'1_30fps_to_240fps/3/0/input/00008.jpg'
    VFI_function(first, second, dir, out, 3+j%2)


1st ok
1st ok
1st ok
1st ok
1st ok
2nd ok with 0
2nd ok with 8
2nd ok with 16
2nd ok with 24
2nd ok with 32
2nd ok with 40
2nd ok with 48
2nd ok with 56
2nd ok with 64
2nd ok with 72
2nd ok with 80
2nd ok with 88
2nd ok with 0
2nd ok with 8
2nd ok with 16
2nd ok with 24
2nd ok with 32
2nd ok with 40
2nd ok with 48
2nd ok with 56
2nd ok with 64
2nd ok with 72
2nd ok with 80
2nd ok with 88
3rd ok with 0
4th ok with 10
3rd ok with 20
4th ok with 30
3rd ok with 40
4th ok with 50
3rd ok with 60
4th ok with 70
3rd ok with 0
4th ok with 10
3rd ok with 20
4th ok with 30
3rd ok with 40
4th ok with 50
3rd ok with 60
4th ok with 70
